### Build model and test follow operation

In [2]:
import os
import sys 
from pathlib import Path 

%load_ext autoreload
%autoreload 2


base_dir = Path(os.getcwd()).parents[0]
sys.path.append(str(base_dir))

from utils import data_utils

### Load Triplets

In [3]:
data_dir = base_dir/'data'
kg_path = data_dir/'kb.txt'
os.listdir(data_dir)
assert kg_path.exists()

triplets, entity_to_idx, relation_to_idx, idx_to_entity, idx_to_relation = data_utils.load_triplets(kg_path)
num_entities = len(entity_to_idx)

num entities: 43234
num relations: 9
num triplets  134741


### Load question transformer

In [ ]:
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')
model = AutoModel.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

### Load dataset 

In [ ]:
import warnings

qa_dir = data_dir/'1-hop-20220731T054004Z-001/1-hop/vanilla'
qa_paths = os.listdir(qa_dir)

qa_path = qa_dir/qa_paths[0]

print(qa_path)
val_pairs = data_utils.load_qa_pairs(qa_path)
data_utils.santity_check(val_pairs, entity_to_idx)

In [ ]:
# [len(row[2]) for row in val_pairs]

### Tokenize dataset

In [ ]:
# # Tokenize sentences
sentences = [row[0] for row in val_pairs]
val_tokens = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')


### Create Torch Dataset

In [ ]:
class QADataset(Dataset):
    def __init__(self, qa_pairs, q_tokens, entity_to_idx):
        self.qa_pairs = qa_pairs
        self.entity_to_idx = entity_to_idx
        self.q_tokens = q_tokens

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        pad = 5
        token_sample = {key: self.q_tokens[key][idx, :] for key in self.q_tokens}
        
        head_entities = self.qa_pairs[idx][2]
        head_entities = [self.entity_to_idx[entity] for entity in head_entities]
                
        # padded
        head_entities = self.pad_entity(head_entities, pad=pad)
        head_entities = torch.LongTensor(head_entities)
        tail_entities = self.qa_pairs[idx][1]
        tail_entities = [self.entity_to_idx[entity] for entity in tail_entities]
        
        tail_entities = self.pad_entity(tail_entities, pad=pad)
        tail_entities = torch.LongTensor(tail_entities)
        
        return token_sample, head_entities, tail_entities
        
    
    def pad_entity(self, list1, pad=5):
        # max length should be this 
        if len(list1) > pad:
            list1 = list1[:pad]
        
        assert len(list1) > 0
        
        list1 = list1 + [-1] * (pad-len(list1))
        return list1
        
        
val_dataset = QADataset(val_pairs, val_tokens, entity_to_idx)


dl = DataLoader(val_dataset, batch_size=10)

for batch in dl:
    break
    
batch[0]['input_ids'].shape

In [ ]:
## Test forward pass of transfomer 

with torch.no_grad():
    out = model(**batch[0])
    out.pooler_output

### Create differentiable KG

In [ ]:
subject_matrix, rel_matrix, object_matrix = data_utils.create_differentiable_kg(triplets, entity_to_idx, relation_to_idx)
object_matrix = torch.transpose(object_matrix, 0, 1)


### Create model

In [ ]:
import models_onehop

net = models_onehop.GNNLightning(model, subject_matrix, rel_matrix, object_matrix)

### Test Follow Operation

In [ ]:
# this model assumes only 1 subject exists
subject_ids = batch[1][:, 0]
# print(subject_ids)
subject_vector = torch.zeros(batch[1].shape[0], num_entities)
subject_vector[range(0, subject_vector.shape[0]), subject_ids] = 1
subject_vector = torch.transpose(subject_vector, 0, 1)

In [ ]:
# def create_sparse_subject():
#     bs = batch[1].shape[0]
#     bs_ids = torch.Tensor(range(0, bs))
#     indices = torch.cat([subject_ids[None, :], bs_ids[None, :]])
#     values = torch.FloatTensor([1] * len(bs_ids))
#     subject_vector = torch.sparse_coo_tensor(indices, values, size=(len(entity_to_idx), bs))
    
#     return subject_vector

# # subject_vector = create_sparse_subject()
# subject_vector = torch.transpose(subject_vector, 0, 1)

In [ ]:
print(subject_vector.shape, subject_matrix.shape)
bs = batch[1].shape[0]
relation = torch.randn(bs, 9)



print('relation: ', relation.shape)
print('subject_vector:', subject_vector.shape)
print('subject_matrix:', subject_matrix.shape)
print('rel_matrix :', rel_matrix.shape)
print('object_matrix1:', object_matrix.shape)

out = net.follow(subject_vector, relation, subject_matrix, rel_matrix, object_matrix)
print(out.shape)

### Perform Follow operation on entity and get results 

In [ ]:
from collections import Counter

def get_sample(id_num, triplets):
    data = list(filter(lambda x: x[0] == id_num, triplets))
        
    # get the relation that leads to multiple entities 
    relation_name = Counter([x[1] for x in data]).most_common(5)[0][0]
    
    objects = []
    for row in data: 
        if row[1] == relation_name:
            objects.append(row[2])
            
    print('num objects ', len(objects))
    return relation_name, objects

subject_id = 0
relation_id, object_ids = get_sample(subject_id, triplets)

def get_entity_names(head_id, rel_id, object_ids):
    head_name = idx_to_entity[head_id]
    rel_name = idx_to_relation[rel_id]
    
    object_name = [idx_to_entity[idx] for idx in object_ids]
    
    print(f'Head {head_name} with relation {rel_name} leads to {object_name}')
    
  
print(f'For head_id {subject_id}, these are the expected results')
get_entity_names(subject_id, relation_id, object_ids)

In [ ]:
import pdb

def get_subject_relation_vector(subject_id, relation_id):
    subject_vector = torch.zeros(1, len(entity_to_idx))
    subject_vector[0, subject_id] = 1 
    
    relation_vector = torch.zeros(1, len(relation_to_idx))
    relation_vector[0, relation_id] = 1 
    
    return subject_vector, relation_vector 

subject_vector, relation_vector = get_subject_relation_vector(subject_id, relation_id)
subject_vector = torch.transpose(subject_vector, 0, 1) # required size

out = net.follow(subject_vector, relation_vector, subject_matrix, rel_matrix, object_matrix)

def interpred_follow_output(out, threshold=0.5):
    """
    Interpret the follow result from the model
    """
    bs, num_entities = out.shape 
        
    outputs_names = []
    output_ids = []
    for idx in range(bs):
        object_probs = out[idx, :]
        
        # set all values less than threshold to be 0
        condition = object_probs >= threshold
        object_preds = object_probs.where(condition, torch.tensor(0.))
        # get indices where value is non zero (true preds)
        pred_object_ids = object_preds.nonzero(as_tuple=True)[0]
        
        pred_object_names = [idx_to_entity[object_id.item()] for object_id in pred_object_ids]
        outputs_names.append(pred_object_names)
        output_ids.append(pred_object_ids.tolist())
            
    return outputs_names, output_ids
        
object_names, object_ids = interpred_follow_output(out, threshold=0.5)  
get_entity_names(subject_id, relation_id, object_ids[0])

### Test forward pass

In [ ]:
def create_subject_vector(subject_ids, num_entities):
    # print(subject_ids)
    batch_size = subject_ids.shape[0]
    subject_vector = torch.zeros(batch_size, num_entities)
    subject_vector[range(0, subject_vector.shape[0]), subject_ids] = 1
    subject_vector = torch.transpose(subject_vector, 0, 1)
    
    return subject_vector

In [ ]:
subject_ids = batch[1]
print(subject_ids.shape)
# all only have one subjects
subject_ids = subject_ids[:, 0]
num_entities = subject_matrix.shape[0]
subject_vector = create_subject_vector(subject_ids, num_entities)

for idx, element in enumerate(subject_ids):
    assert subject_vector[element, idx] == 1


In [ ]:
net = models_onehop.GNNLightning(model, subject_matrix, rel_matrix, object_matrix)

In [ ]:
with torch.no_grad():
    trans_input = batch[0]
    subject_ids = batch[1]
    subject_ids = subject_ids[:, 0]
    batch_size = subject_ids.shape[0]
    
    subject_vector = create_subject_vector(subject_ids, num_entities)
    
    num_entities = subject_matrix.shape[1]
    
    # check shape
    assert num_entities == len(entity_to_idx)
    
    subject_vector = create_subject_vector(subject_ids, num_entities)    
    out = net.follow(subject_vector, relation, subject_matrix, rel_matrix, object_matrix)
    
    out_model = net(trans_input, subject_vector)
    
print('done')

In [ ]:
# def prepare_subject_vector():
    

In [ ]:
# subject_ids = subject_ids[:, 0]
# batch_size = subject_ids.shape[0]

# subject_vector = create_subject_vector(subject_ids, num_entities)